In [257]:
import cv2
import numpy as np

# Load the scene and its depth map
scene = cv2.imread('img6.png')
depth_map = cv2.imread('6_img_depth.png', cv2.IMREAD_GRAYSCALE)  # Assuming a grayscale depth map

In [258]:
# Load the object with transparency (alpha channel)
object_img = cv2.imread('Sports-Ball-Transparent.png', cv2.IMREAD_UNCHANGED)

object_img = cv2.resize(object_img, (50, 50))

# cv2.imshow('Integrated Scene', depth_map)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [259]:
# Initialize a list to store clicked locations
clicked_locations = []

def click_event(event, x, y, flags, param):
    # If the left mouse button was clicked, record the position
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked_locations.append((x, y))

# Set the mouse callback function for the window
cv2.namedWindow('Select Locations')
cv2.setMouseCallback('Select Locations', click_event)

# Display the image and wait for user to select locations
cv2.imshow('Select Locations', scene)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [260]:
# Constants for scaling
max_depth = 255
min_scale = 0.5
max_scale = 5

# Process each clicked location
for location in clicked_locations:
    object_location_x, object_location_y = location
    depth_value = depth_map[object_location_y, object_location_x]
    scale_factor = (depth_value / max_depth) * (max_scale - min_scale) + min_scale
    resized_object = cv2.resize(object_img, (int(object_img.shape[1] * scale_factor), int(object_img.shape[0] * scale_factor)))

    # Insert the object into the scene at the clicked location
    scene_height, scene_width = scene.shape[:2]
    for i in range(resized_object.shape[0]):
        for j in range(resized_object.shape[1]):
            if resized_object[i, j, 3] > 0:  # Check for alpha channel transparency
                y_index = object_location_y + i
                x_index = object_location_x + j
                # Check if the current indices are within the scene's boundaries
                if x_index < scene_width and y_index < scene_height:
                    scene[y_index, x_index, :] = resized_object[i, j, :3]

# Display the final scene
cv2.imshow('Integrated Scene', scene)
cv2.waitKey(0)
cv2.destroyAllWindows()